In [13]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

val spark = SparkSession.builder
  .appName("User Rating History Partitioning")
  .master("yarn")
  .getOrCreate()

val sc = spark.sparkContext
val ratingsFilePath = "gs://srinija/archive/rating.csv"

val ratingsDF = spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(ratingsFilePath)



lastException = null
spark = org.apache.spark.sql.SparkSession@4c4771e0
sc = org.apache.spark.SparkContext@3e00468f
ratingsFilePath = gs://srinija/archive/rating.csv
ratingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [14]:
val validRatingsDF = ratingsDF.filter(
  col("userId").isNotNull &&
  col("movieId").isNotNull &&
  col("rating").isNotNull &&
  col("timestamp").isNotNull
)

validRatingsDF.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



validRatingsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [15]:
val uratingsDF = validRatingsDF.select("userId", "movieId", "rating")
uratingsDF.printSchema()

uratingsDF = [userId: int, movieId: int ... 1 more field]


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



[userId: int, movieId: int ... 1 more field]

In [16]:
val trimmedDF = uratingsDF.limit(100000)

trimmedDF = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [17]:
val outputPath = "hdfs:///user/day_16_17/case_study_2"
trimmedDF.coalesce(1).write.partitionBy("userId").format("parquet").mode("overwrite").save(outputPath)

outputPath = hdfs:///user/day_16_17/case_study_2


hdfs:///user/day_16_17/case_study_2

In [18]:
val path = "hdfs:///user/day_16_17/case_study_2/userId=1"
spark.read.option("basePath", outputPath).parquet(path).select("userId","movieId","rating").distinct().show()


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   7438|   4.0|
|     1|   4011|   4.0|
|     1|   2173|   4.0|
|     1|   5816|   4.0|
|     1|   4911|   4.0|
|     1|     50|   3.5|
|     1|    293|   4.0|
|     1|   1089|   3.5|
|     1|   2118|   4.0|
|     1|   6539|   4.0|
|     1|   8690|   3.5|
|     1|   7757|   4.0|
|     1|    367|   3.5|
|     1|   3476|   3.5|
|     1|   1358|   4.0|
|     1|   4105|   3.5|
|     1|   5952|   5.0|
|     1|   3499|   4.0|
|     1|   1350|   3.5|
|     1|   1222|   3.5|
+------+-------+------+
only showing top 20 rows



path = hdfs:///user/day_16_17/case_study_2/userId=1


hdfs:///user/day_16_17/case_study_2/userId=1

In [19]:
spark.stop()